In [1]:
import bs4 as bs
import requests
import pandas as pd
import datetime as dt
import os
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from tiingo import TiingoClient

# auth 
yourtoken = os.getenv("TIINGO_API_KEY")

config = {}
config['session'] = True
config['api_key'] = yourtoken

client = TiingoClient(config)

# since requests doesn't work correctly on JS objects, we will just manually get the DOW30 tickers + DJI
def DJIA_tickers() -> list[str]:
    return ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'XOM', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIA', 'SPY']

# TODO: try this with PP - pricing power 
# TODO: try this with QQQ - technologies 
# TODO: try this with GUNR / other stable ETFs

In [2]:
def dataReader(ticker, d_start, d_end, frequency):
    d_end = d_end.strftime("%Y-%m-%d")
    d_start = d_start.strftime("%Y-%m-%d")
    print("Getting Data from Tiingo")
    df = client.get_dataframe(ticker, startDate=d_start, endDate=d_end, frequency=frequency)
    # print(f"head: {df.head()}")
    #print(f"columns: {df.columns}")
    # print(f"shape: {df.shape}")
    #print(f"index: {df.index}")
    # print(f"index name: {df.index.name}")
    #print(f"info: {df.info()}")
    
      # Check for existance of Adj Close column
      # If not, check for existance of Close column
      # If not -> throw error
      # If no Adj Close, but Close, copy Close to Adj close

    if not "adjClose" in df.columns:
        if "close" in df.columns:
                df["adjClose"] = df["close"]
        else:
                print("Error: No Close information")
        return
      
    # Convert ISO date format to Pandas DateTime
    #df["date"] = pd.to_datetime(df["date"])
    # Align Column Names to previous DataReader names
    df.index = pd.to_datetime(df.index)
    print(f"index: {df.index}")
    #print(f"info: {df.info()}")

    df = df.rename(
        columns={
        'date': "Date",
        "open": "Open",
        "adjClose": "Adj Close",
        "volume": "Volume",
        "high": "High",
        "low": "Low",
        "close": "Close",
        }
    )
    # print(df)
    #a = df.set_index(["Date"])
    # print(a)
    return df

In [3]:
# def DJIA_tickers():
# 	'''

# 	Parses Slickcharts Webpage to obtain the tickers for each current listed stock in the DJIA.

# 	'''

# 	resp = requests.get('https://money.cnn.com/data/dow30/') # https://www.cnbc.com/dow-30/
# 	soup = bs.BeautifulSoup(resp.text, "lxml")
# 	table = soup.find('table', {'class': 'wsod_dataTable wsod_dataTableBig'}) # BasicTable-table
# 	tickers = []
# 	for row in table.findAll('tr')[1:]:
# 		ticker = row.findAll('td')[0].text
# 		ticker = ticker.split('\xa0', 1)[0]
# 		tickers.append(ticker)

# 	return tickers

# DJIA_tickers()
# print(DJIA_tickers())

In [4]:
def API_data():
	'''

	Uses Tiingo API to access all the historical data for the current stocks listed in the DJIA.

	'''
	# DJIA = 'DIA'
	tickers = DJIA_tickers()
	# tickers.append(DJIA)

	if not os.path.exists('stocks_DJIA'):
		os.makedirs('stocks_DJIA')

	start = dt.datetime(2000,1,1)
	end = dt.datetime.now()
	print(f"tickers: {tickers}")
	
	for ticker in tickers:
		if not os.path.exists('stock_DJIA/{}.csv'.format(ticker)):
			df = dataReader(ticker, start, end, frequency='daily')
			df.to_csv('stocks_DJIA/{}.csv'.format(ticker))
			print(df)
		else:
			print('Information already acquired for {}'.format(ticker))

print(f"api_data: {API_data()}")

tickers: ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'XOM', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIA', 'SPY']
Getting Data from Tiingo
index: DatetimeIndex(['2000-01-03 00:00:00+00:00', '2000-01-04 00:00:00+00:00',
               '2000-01-05 00:00:00+00:00', '2000-01-06 00:00:00+00:00',
               '2000-01-07 00:00:00+00:00', '2000-01-10 00:00:00+00:00',
               '2000-01-11 00:00:00+00:00', '2000-01-12 00:00:00+00:00',
               '2000-01-13 00:00:00+00:00', '2000-01-14 00:00:00+00:00',
               ...
               '2023-01-12 00:00:00+00:00', '2023-01-13 00:00:00+00:00',
               '2023-01-17 00:00:00+00:00', '2023-01-18 00:00:00+00:00',
               '2023-01-19 00:00:00+00:00', '2023-01-20 00:00:00+00:00',
               '2023-01-23 00:00:00+00:00', '2023-01-24 00:00:00+00:00',
               '2023-01-25 00:00:00+00:00', '2023-01-2

In [5]:
def compile_data():
	'''

	Creates a dataframe with the compiled adjusted closing for all the stocks in the DJIA.

	'''

	tickers = DJIA_tickers()

	main_df = pd.DataFrame()

	for count,ticker in enumerate(tickers):
		df = pd.read_csv('stocks_DJIA/{}.csv'.format(ticker))
		df.set_index('date', inplace= True)

		df.rename(columns = {'Adj Close': ticker}, inplace=True)

		df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)

		if main_df.empty:
			main_df = df
		else:
			main_df = main_df.join(df, how='outer')

		if count % 10 == 0:
			print(count)

	print(main_df.head())
	main_df.to_csv('DJIA_adjcloses.csv')

print(f"compiling data...")
compile_data()


compiling data...
0
10


C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitF

20


C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitF

30
                                 MMM        AXP       AMGN      AAPL  \
date                                                                   
2000-01-03 00:00:00+00:00  25.853839  33.546495  47.210729  0.850782   
2000-01-04 00:00:00+00:00  24.826588  32.277169  43.602791  0.779034   
2000-01-05 00:00:00+00:00  26.092161  32.111836  44.960456  0.790435   
2000-01-06 00:00:00+00:00  27.612492  32.111836  45.853064  0.722032   
2000-01-07 00:00:00+00:00  28.160359  32.560504  51.006190  0.756233   

                                  BA        CAT        CVX       CSCO  \
date                                                                    
2000-01-03 00:00:00+00:00  25.897581  13.183454  17.994559  38.101663   
2000-01-04 00:00:00+00:00  25.858918  13.012663  17.994559  35.951616   
2000-01-05 00:00:00+00:00  27.669622  13.503349  18.384014  36.233589   
2000-01-06 00:00:00+00:00  27.746948  13.996746  19.096223  35.246682   
2000-01-07 00:00:00+00:00  28.552421  14.452189  19.43

C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'], 1, inplace=True)
C:\Users\rohit\AppData\Local\Temp\ipykernel_26712\1414809035.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitF